In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from astropy import wcs
from astropy.nddata import Cutout2D
from astropy import coordinates
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
import matplotlib.gridspec as gridspec
import aplpy
import os.path
from photutils import SkyCircularAperture
from photutils import aperture_photometry

In [2]:
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/chandra_7ms/big_table.fits'
big_table = fits.open(path)[1].data
visual = big_table.field('Visual_Flag')
usable_b = big_table.field('GOODS_b_Usable')
usable_v = big_table.field('GOODS_v_Usable')
usable_i = big_table.field('GOODS_i_Usable')
usable_z = big_table.field('GOODS_z_Usable')
usable = [usable_b, usable_v, usable_i, usable_z]
aperture = big_table.field('Aperture') #arcsec diameter
minarea = big_table.field('Min_Area') #pixel
spitzer = big_table.field('Spitzer_Aper_Flag') #arcsec diameter

a = ['goods_b', 'goods_v', 'goods_i', 'goods_z']
b = ['goods_b', 'goods_v', 'goods_i', 'goods_z']
c = ['goods', 'goods', 'goods', 'goods']
aper_pixel = 1.
min_area = 1.
for j in range(len(a)):
    for i in range(big_table.size):
        if (visual[i] > 3) & (usable[j][i] > 0):
            aper_pixel = aperture[i] / 0.03
            min_area = minarea[i]
            n_thresh = 64.
            min_cont = 0.0001
            if aper_pixel != 0.:
                conf_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/' + c[j] + '/' + b[j] + '.conf'
                im_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + '.fits'
                dualim_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + '.fits,/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + '.fits'
                cat_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/test_goods/cat_' + a[j] + str(i+1) + '.cat'
                rms_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + 'rms.fits'
                whtrms_path = ' /Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + 'rms.fits,/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/' + c[j] + '/cutout_' + a[j] + str(i+1) + 'wht.fits'
                check_path = ' /Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/test_goods/cat_' + a[j] + str(i+1) + '_aper.fits,/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/test_goods/cat_' + a[j] + str(i+1) + '_bkg.fits,/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/test_goods/cat_' + a[j] + str(i+1) + '_bkg_rms.fits' 
                path = 'sex -c' + conf_path + im_path + ' -CATALOG_NAME' + cat_path + ' -WEIGHT_TYPE MAP_RMS -WEIGHT_GAIN N -WEIGHT_IMAGE' + rms_path + ' -DETECT_MINAREA ' + str(min_area) + ' -CHECKIMAGE_NAME' + check_path + ' -BACK_SIZE 90 -DEBLEND_MINCONT ' + str(min_cont) + ' -DEBLEND_NTHRESH ' + str(n_thresh) + ' -PHOT_APERTURES 2.93,4.17,5.87,8.33,11.77,16.67,23.57,33.33,47.13,66.67,94.27 -PARAMETERS_NAME /Users/justin/Documents/Master_Thesis/data/catalogues/goods/test_goods.param -MASK_TYPE NONE -CHECKIMAGE_TYPE APERTURES,BACKGROUND,BACKGROUND_RMS'
                os.system(path)
        if (visual[i] > 3) & (usable[j][i] == 0):
            print('Problem at source' + str(i+1) + ', band' + b[j])


In [9]:
path = '/Users/justin/Documents/Master_Thesis/data/surveys/cutouts/goods/cutout_goods_i137.fits'
im = fits.open(path)[0]
position = SkyCoord(53.0219789*u.deg, -27.7927204*u.deg, frame='icrs')
circaper = SkyCircularAperture(position, r=0.8*u.arcsec)
phot_table = aperture_photometry(im, circaper)
print(phot_table['aperture_sum'][0])
path = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/test_goods/cat_goods_i391_bkg.fits'
im = fits.open(path)[0]
position = SkyCoord(53.0924360*u.deg, -27.8268658*u.deg, frame='icrs')
circaper = SkyCircularAperture(position, r=1.6*u.arcsec)
phot_table = aperture_photometry(im, circaper)
print(phot_table['aperture_sum'][0])

9.53164952102
3.60268421594


In [4]:
search_radius=0.4
search_radius2=0.2
d = ['b', 'v', 'i', 'z']
goods_aper_pix = [2.93,4.17,5.87,8.33,11.77,16.67,23.57,33.33,47.13,66.67,94.27]
goods_aper = np.zeros(len(goods_aper_pix))
se_m = np.zeros(len(goods_aper_pix))
se_me = np.zeros(len(goods_aper_pix))
se_f = np.zeros(len(goods_aper_pix))
se_fe = np.zeros(len(goods_aper_pix))
goods_m = np.zeros(len(goods_aper_pix))
goods_me = np.zeros(len(goods_aper_pix))
goods_f = np.zeros(len(goods_aper_pix))
goods_fe = np.zeros(len(goods_aper_pix))
for i in range(len(goods_aper_pix)):
    goods_aper[i] = goods_aper_pix[i] * 0.03 #diameter arcsec
for j in range(len(d)):
    path='/Users/justin/Documents/Master_Thesis/data/catalogues/goods/h_goods_s' + d[j] + '_r2.0z_cat_nohdr.txt'
    t2 = pd.read_csv(path, delim_whitespace=True, header=None)
    for i in range(big_table.size):
        if (visual[i] > 3) & (usable[j][i] > 0):
            ra = big_table.field('CANDELS_160_RA')[i]
            dec = big_table.field('CANDELS_160_Dec')[i]
            path = '/Users/justin/Documents/Master_Thesis/data/catalogues/cutouts/test_goods/cat_' + a[j] + str(i+1) + '.cat'
            t1 = pd.read_csv(path, delim_whitespace=True, header=None)
            #9-19,20-30,31-41,42-52 + 3-4
            radius = ((t1[3]-ra).pow(2)+(t1[4]-dec).pow(2)).pow(0.5)
            n=-1
            for k in range(len(t1)):
                if radius[k] < search_radius /3600.:
                    if n == -1:
                        n = k
                    else:
                        if radius[k] < radius[n]:
                            n = k
            if n != -1:
                radius=((t2[1]-t1[3][n]).pow(2)+(t2[2]-t1[4][n]).pow(2)).pow(0.5)
                m=-1
                for l in range(len(t2)):
                    if radius[l] < search_radius2 /3600.:
                        if m == -1:
                            m = l
                        else:
                            if radius[l] < radius[m]:
                                m = l
                if m != -1:
                    for h in range(len(goods_aper)):
                        se_m[h]=t1[9+h][n]
                        se_me[h]=t1[20+h][n]
                        se_f[h]=t1[31+h][n]
                        se_fe[h]=t1[42+h][n]
                        goods_m[h]=t2[49+h][m]
                        goods_me[h]=t2[60+h][m]
                        goods_f[h]=t2[71+h][m]
                        goods_fe[h]=t2[82+h][m]
                        print(goods_fe[h]/se_fe[h])
                    f = plt.figure()
                    ax = f.add_subplot(111)
                    ax.plot(goods_aper, se_m, 'b-', goods_aper, se_me, 'b--', goods_aper, goods_m, 'r-', goods_aper, goods_me, 'r--')
                    plt.yscale('linear')
                    path = '/Users/justin/Documents/Master_Thesis/presentation/plots/test_goods_aper/m' + d[j] + str(i+1) + '.pdf'
                    plt.savefig(path, format='pdf', dpi=1000)
                    plt.clf()
                    f = plt.figure()
                    ax = f.add_subplot(111)
                    ax.plot(goods_aper, se_f, 'b-', goods_aper, se_fe, 'b--', goods_aper, goods_f, 'r-', goods_aper, goods_fe, 'r--')
                    plt.yscale('log')
                    path = '/Users/justin/Documents/Master_Thesis/presentation/plots/test_goods_aper/f' + d[j] + str(i+1) + '.pdf'
                    plt.savefig(path, format='pdf', dpi=1000)
                    plt.clf()
                
                

            

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (34,38,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1.31441038668
1.29017799356
1.31248369769
1.25728471984
1.19528729582
1.16597333759
1.14873495163
1.13710022564
1.12722180834
1.13291054046
1.12528545168
1.28265983932
1.22773869009
1.19815694973
1.1603526632
1.15395813439
1.15856538708
1.13046653114
1.12972107815
1.12480708786
1.12228927381
1.12110960865
1.30001757052
1.22000945698
1.17352704141
1.1659515601
1.16036154656
1.13745398923
1.13093772255
1.12594058803
1.12348702602
1.12116206607
1.11938703991
1.13880685862
1.15145287028
1.18496680584
1.18002180722
1.17370184529
1.15912058964
1.16029466455
1.14673017726
1.12716784638
1.11745230299
1.1182324093
1.14380356987
1.16720081281
1.16963512871
1.15238831251
1.14037633934
1.13244295455
1.12320671883
1.12120502792
1.1243518245
1.12400550669
1.12309202865
1.18671967188
1.18309568021
1.14624364023
1.15659733781
1.13921675353
1.12412815414
1.12491103598
1.12248852484
1.12259959845
1.12013298484
1.11621584733
1.21510112114
1.20902229789
1.15765658337
1.13793825671
1.11441478988
1.10822241

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


1.36108137724
1.26524294792
1.20010230442
1.15137366431
1.12525257195
1.10784572332
1.1015405046
1.09229473791
1.08779106074
1.08692324532
1.08361721565
1.32480495906
1.27144766778
1.2167263429
1.16765209101
1.13280920158
1.11438864424
1.12394957804
1.18512610655
1.32232256198
1.53842845614
1.80899131389
1.12774799051
1.10232591541
1.10905129316
1.11203760277
1.10262582445
1.08589184114
1.06996467877
1.06620467726
1.07230165937
1.0721376806
1.07282202555
1.24515261593
1.24766469869
1.18685960197
1.17124995365
1.14176441167
1.12991027425
1.11326235199
1.1021633085
1.09456834027
1.08976632248
1.08748045671
1.11845068257
1.1088735422
1.09198551899
1.09241006989
1.08675305826
1.08013423679
1.07855469952
1.0734727789
1.07438767707
1.0736578812
1.07343725133
1.18228502477
1.11909940494
1.10194824828
1.09931165133
1.10317333647
1.09517840782
1.09008326653
1.08704686838
1.08561343766
1.0847761131
1.08340587546
1.27224527185
1.21218737673
1.17778866432
1.14889116611
1.12013529873
1.10035223644


In [5]:
#
print(1.0857/0.9214101*((0.9214101/7200)+(94.27**2/4*3.14*0.0008637**2))**0.5)
print((1/500000)**0.5)

0.08604059679097383
0.001414213562373095
